In [1]:
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

login(token="replace with your token")
model_name = 'meta-llama/Llama-3.2-1B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

/workspace/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
! pip install peft
! pip install -U bitsandbytes

In [2]:
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map='auto', trust_remote_code=True)
model.resize_token_embeddings(len(tokenizer))
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.05, bias='none', task_type="CAUSAL_LM")
model = get_peft_model(model, peft_config)

In [3]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 163.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 182.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 279.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0


In [3]:
from datasets import load_dataset

# could consider using a Dataloader to set up num_workers
# example: train_loader = DataLoader(train_dataset,
#    batch_size=16,      # Adjust based on your GPU memory
#    num_workers=4,     # Use 4 worker processes for data loading
#    pin_memory=True     # Optional: speeds up data transfer to GPU)

dataset = load_dataset('Salesforce/wikitext', 'wikitext-103-v1')

Generating validation split: 100%|██████████| 3760/3760 [00:00<00:00, 471453.26 examples/s]


In [4]:
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    batch = tokenizer(examples["text"], padding="max_length", max_length=128, truncation=True)
    batch["labels"] = batch["input_ids"]
    return batch

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 3760/3760 [00:00<00:00, 11188.75 examples/s]


In [5]:
! pip install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 159.9 MB/s eta 0:00:00


In [6]:
import wandb

wandb.login()  # Log in directly without setting env variable
wandb.init(project='llama-training', entity='fjiang7-ucsd')

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fjiang7 (fjiang7-ucsd). Use `wandb login --relogin` to force relogin


In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="steps",
    eval_steps=30, # maybe should set bigger (200)
    learning_rate=2e-5,
    per_device_train_batch_size=16, #maybe try larger batch size
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    gradient_accumulation_steps=2,
    report_to="wandb",
    logging_dir='./logs',  # directory for storing logs
    logging_steps=50,
    save_steps=500,
    fp16=True,
)

/workspace/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
! pip install trl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
)

/workspace/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/workspace/miniconda3/lib/python3.12/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [10]:
train_stats = trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/workspace/miniconda3/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
30,No log,3.423285
60,3.300800,3.324385
90,3.300800,3.232640
120,3.139900,3.154700
150,3.001300,3.089830
180,3.001300,3.037256
210,2.960700,2.991738
240,2.960700,2.955035
270,2.908900,2.925846
300,2.846000,2.901701


/workspace/miniconda3/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/workspace/miniconda3/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/workspace/miniconda3/lib/python

KeyboardInterrupt: 

In [11]:
wandb.finish()

eval/loss,█▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/runtime,▃▇▂▄▅▂▄▃▇▅▅▅▄▄▃▅▁▆▄▇▄█▄▃▅▂▃▂▂▄▇▃▃▃▅▇▂▆▁▂
eval/samples_per_second,▆▄▆▅▄▇▇▇▆▆▅▁▅▆▆▅█▅▇▃▃▅▆▃▆▇▆▆▆▆▇▆▄▃▆▄▃▇▄▇
eval/steps_per_second,▂▇▄▅▃▇▅▆▆▆▄▃▄▁▅▆▃█▃▇▃▅▁▅▅▆▆▆▄▇▆▆▄▇▆▆▆▃▁▂
train/epoch,▁▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇█████
train/global_step,▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇███
train/grad_norm,▁▁▃▄▃▄▄▃▅▄▆▆▃▇▄█▃▆▄▄▄▄▄▅▃▃▄▅▅▃▂▄▅█▃▄▆▅ ▄
train/learning_rate,████▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▆▅▄▄▂▃▂▂▂▂▃▂▂▂▂▂▂▂▂▁▂▂▂▂▁▁▂▁▂▂▂▁▁▁▁▁▁▁▁
eval/loss,2.71482
eval/runtime,40.2794
